In [ ]:
import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Conv2D,Input,MaxPooling2D,UpSampling2D
from tensorflow.python.keras.layers.merge import concatenate
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
# (128*128) 
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

def data_generator(data_dir,mode,target_size=(128,512),batch_size=32,shuffle=True):
    
    for imgs in ImageDataGenerator().flow_from_directory(
    directory=data_dir,
    classes=[mode],
    class_mode=None,
    color_mode='rgb',
    target_size=target_size,
    batch_size=batch_size,
    shuffle=shuffle
    ):
       
        y=imgs[:,0:128,128:256]/255
        x=np.zeros((y.shape[0],128,128,7))

        x[:,0:128,0:128,:3]=imgs[:,0:128,0:128]/255
        x[:,0:128,0:128,3:6]=imgs[:,0:128,256:384]/255
        x[:,0:128,0:128,6]=imgs[:,0:128,384:512,0]/255
          
        yield x,y

        
BATCH_SIZE=32


# color+normal+depth
DATA_DIR='D:/training_model/20190606/'
train_data_generator=data_generator(DATA_DIR,'sponza',batch_size=BATCH_SIZE)


In [ ]:
tbCallBack = TensorBoard(log_dir='./logs', 
                 histogram_freq=0, 
                 batch_size=32,     
                 write_graph=True, 
                 write_grads=True, 
                 write_images=True,
                 embeddings_freq=0, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)

In [ ]:
from tensorflow.image import ssim
def ssim_fun(y_true,y_pred):
    s=ssim(y_true,y_pred,1)
    return s

def psnr(y_true,y_pred):
    return -10*K.log(
    K.mean(K.flatten((y_true-y_pred))**2)
    )/np.log(10)

inputs=Input((None,None,7),name='in')
conv1=Conv2D(32,kernel_size=(3,3),strides=1,activation='relu',padding="same",name='conv1')(inputs)
pool1=MaxPooling2D((2,2),name='pool1')(conv1)

conv2=Conv2D(48,kernel_size=(3,3),strides=1,activation='relu',padding="same",name='conv2')(pool1)
pool2=MaxPooling2D((2,2),name='pool2')(conv2)

conv3=Conv2D(56,kernel_size=(3,3),strides=1,activation='relu',padding="same",name='conv3')(pool2)
pool3=MaxPooling2D((2,2),name='pool3')(conv3)


unpool2=concatenate([UpSampling2D(size=(2, 2))(pool3),pool2],name='unpool2')
conv4=Conv2D(56,kernel_size=(3,3),strides=1,activation='relu',padding="same",name='conv4')(unpool2)
conv4=Conv2D(56,kernel_size=(3,3),strides=1,activation='relu',padding="same",name='conv5')(conv4)

unpool1=concatenate([UpSampling2D(size=(2, 2))(conv4),pool1],name='unpool1')
conv5=Conv2D(48,kernel_size=(3,3),strides=1,activation='relu',padding="same",name='conv6')(unpool1)
conv5=Conv2D(48,kernel_size=(3,3),strides=1,activation='relu',padding="same",name='conv7')(conv5)

unpool0=concatenate([UpSampling2D(size=(2, 2))(conv5),inputs],name='inputs')
conv6=Conv2D(48,kernel_size=(3,3),strides=1,activation='relu',padding="same",name='conv8')(unpool0)
conv6=Conv2D(32,kernel_size=(3,3),strides=1,activation='relu',padding="same",name='conv9')(conv6)

conv7=Conv2D(3,kernel_size=(3,3),strides=1,activation='relu',padding="same",name='out')(conv6)


model=Model(inputs=inputs,outputs=conv7)
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=[psnr,ssim_fun]
)


model.fit(train_data_generator,steps_per_epoch=32,epochs=100,callbacks=[tbCallBack],shuffle=True)


In [ ]:
from tensorflow.image import ssim
def ssim_fun(y_true,y_pred):
    s=ssim(y_true,y_pred,1)
    return s

def psnr(y_true,y_pred):
    return -10*K.log(
    K.mean(K.flatten((y_true-y_pred))**2)
    )/np.log(10)

model = tf.contrib.keras.models.load_model('./20190606/model_100.h5',custom_objects={"psnr":psnr,"ssim_fun":ssim_fun})


In [ ]:
 
#     testing data path
path_t='C:/Users/NTNU_CGLab/Documents/denoisy/TestData/sponza/point light/1sppImage_20190606.ppm'
path_n='C:/Users/NTNU_CGLab/Documents/denoisy/TestData/sponza/point light/1sppImage_Normal_20190606.ppm'
path_d='C:/Users/NTNU_CGLab/Documents/denoisy/TestData/sponza/point light/1sppImage_Depth_20190606.ppm'


x=np.zeros((1,720,1280,7))

#     get noise
img_x=cv2.imread(path_t)
img_x=img_x[:,:,::-1]
x[0,:,:,:3]=img_x

#     get normal
img_n=cv2.imread(path_n)
img_n=img_n[:,:,::-1]
x[0,:,:,3:6]=img_n

#     get depth
img_d=cv2.imread(path_d)
img_d=img_d[:,:,::-1]
x[0,:,:,6]=img_d[:,:,0]



prds_img=model.predict(x) 
plt.imshow(img_x/255)
plt.show()
plt.imshow(prds_img[0]/255)
plt.show()


# for h in range(prds_img[0].shape[0]):
#     for w in range(prds_img[0].shape[1]):
#         r,g,b=prds_img[0,h,w]/255
#         if r>1:
#             r=1
#         if g>1:
#             g=1
#         if b>1:
#             b=1    
#         prds_img[0,h,w]=r,g,b
        
# plt.imsave("./20190606/denoiser_100_sponza_p", prds_img[0])  


# ground truth
path2='C:/Users/NTNU_CGLab/Documents/denoisy/TestData/sponza/point light/64sppImage_20190606.ppm'


img2 = cv2.imread(path2)
img2 = img2[:,:,::-1]
test_y=np.array((img2[:720,:1280,:]))
test_y=test_y[np.newaxis,:]

loss_function,psnr,ssim=model.evaluate(x/255,test_y/255)
print("loss function:",loss_function)
print("PSNR: ",psnr)
print("SSIM: ",ssim)

  